# 프로젝트: QA bAbI 
스토리를 입력 데이터셋으로 하고 질문에 대한 답변을 스토리에서 찾아 답하는 프로젝트입니다. 
* Dataset:SQuAD, bAbI
    - SQuAD: Reading Comprehension
    - bAbI: Multi-hop Reasoning Dataset => 대화 도중 주고받은 정보를 재활용하여 커뮤니케이션할 수 있는 챗봇을 만들기 위해 필요한 데이터셋 
    
> 주의사항: supporting fact는 웬만하면 학습하지 않는 것이 원칙 

In [1]:
!pip install customized_konlpy

In [2]:
from transformers import pipeline

# 설치 확인 
classifier = pipeline('sentiment-analysis', framework='tf')
classifier('We are very happy to include pipeline into the transformers repository.')

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


[{'label': 'POSITIVE', 'score': 0.9978193640708923}]

## Library

In [3]:
from tensorflow.keras.utils import get_file
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import zipfile
from nltk import FreqDist
from functools import reduce
import os
import re

from transformers import pipeline

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Permute, dot, add, concatenate
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Activation
import matplotlib.pyplot as plt

## Data Load

In [5]:
# 환경에 맞게 경로 수정
file_path = os.getenv('HOME')+'/aiffel/babi_memory_net/data'
file_to_save = file_path + '/e-28-korean.zip'
path = get_file(file_to_save, origin='https://aiffelstaticprd.blob.core.windows.net/media/documents/e-28-korean.zip')
print()
print(file_to_save)
print(path)


/home/aiffel-dj19/aiffel/babi_memory_net/data/e-28-korean.zip
/home/aiffel-dj19/aiffel/babi_memory_net/data/e-28-korean.zip


In [6]:
TRAIN_FILE = os.path.join(file_path, "qa1_single-supporting-fact_train_kor.txt")
TEST_FILE = os.path.join(file_path, "qa1_single-supporting-fact_test_kor.txt")

In [7]:
def show_data(path):
    i = 0
    lines = open(TRAIN_FILE , "rb")
    for line in lines:
        line = line.decode("utf-8").strip()
        lno, text = line.split(" ", 1) # ID와 TEXT 분리
        i = i + 1
        print(line)
        if i == 20:
            break

In [8]:
# 훈련데이터 20개의 문장 출력
show_data(TRAIN_FILE)

1 필웅이는 화장실로 갔습니다.
2 은경이는 복도로 이동했습니다.
3 필웅이는 어디야? 	화장실	1
4 수종이는 복도로 복귀했습니다.
5 경임이는 정원으로 갔습니다.
6 수종이는 어디야? 	복도	4
7 은경이는 사무실로 갔습니다.
8 경임이는 화장실로 뛰어갔습니다.
9 수종이는 어디야? 	복도	4
10 필웅이는 복도로 갔습니다.
11 수종이는 사무실로 가버렸습니다.
12 수종이는 어디야? 	사무실	11
13 은경이는 정원으로 복귀했습니다.
14 은경이는 침실로 갔습니다.
15 경임이는 어디야? 	화장실	8
1 경임이는 사무실로 가버렸습니다.
2 경임이는 화장실로 이동했습니다.
3 경임이는 어디야? 	화장실	2
4 필웅이는 침실로 이동했습니다.
5 수종이는 복도로 갔습니다.


In [9]:
# 테스트데이터 20개의 문장 출력
show_data(TEST_FILE)

1 필웅이는 화장실로 갔습니다.
2 은경이는 복도로 이동했습니다.
3 필웅이는 어디야? 	화장실	1
4 수종이는 복도로 복귀했습니다.
5 경임이는 정원으로 갔습니다.
6 수종이는 어디야? 	복도	4
7 은경이는 사무실로 갔습니다.
8 경임이는 화장실로 뛰어갔습니다.
9 수종이는 어디야? 	복도	4
10 필웅이는 복도로 갔습니다.
11 수종이는 사무실로 가버렸습니다.
12 수종이는 어디야? 	사무실	11
13 은경이는 정원으로 복귀했습니다.
14 은경이는 침실로 갔습니다.
15 경임이는 어디야? 	화장실	8
1 경임이는 사무실로 가버렸습니다.
2 경임이는 화장실로 이동했습니다.
3 경임이는 어디야? 	화장실	2
4 필웅이는 침실로 이동했습니다.
5 수종이는 복도로 갔습니다.


In [10]:
def read_data(dir):
    stories, questions, answers = [], [], [] # 각각 스토리, 질문, 답변을 저장할 예정
    story_temp = [] # 현재 시점의 스토리 임시 저장
    lines = open(dir, "rb")

    for line in lines:
        line = line.decode("utf-8") # b' 제거
        line = line.strip() # '\n' 제거
        idx, text = line.split(" ", 1) # 맨 앞에 있는 id number 분리
        # 여기까지는 모든 줄에 적용되는 전처리

        if int(idx) == 1:
            story_temp = []
        
        if "\t" in text: # 현재 읽는 줄이 질문 (tab) 답변 (tab)인 경우
            question, answer, _ = text.split("\t") # 질문과 답변을 각각 저장
            stories.append([x for x in story_temp if x]) # 지금까지의 누적 스토리를 스토리에 저장
            questions.append(question)
            answers.append(answer)

        else: # 현재 읽는 줄이 스토리인 경우
            story_temp.append(text) # 임시 저장

    lines.close()
    return stories, questions, answers

In [11]:
train_data = read_data(TRAIN_FILE)
test_data = read_data(TEST_FILE)

In [95]:
train_stories, train_questions, train_answers = read_data(TRAIN_FILE)
test_stories, test_questions, test_answers = read_data(TEST_FILE)

In [96]:
print("train 스토리 개수:", len(train_stories))
print("train 질문 개수:", len(train_questions))
print("train 답변 개수:", len(train_answers))
print("test 스토리 개수:", len(test_stories))
print("test 질문 개수:", len(test_questions))
print("test 답변 개수:", len(test_answers))

train 스토리 개수: 10000
train 질문 개수: 10000
train 답변 개수: 10000
test 스토리 개수: 1000
test 질문 개수: 1000
test 답변 개수: 1000


In [97]:
train_stories[3878]

['수종이는 화장실로 뛰어갔습니다.',
 '경임이는 사무실로 갔습니다.',
 '은경이는 부엌으로 이동했습니다.',
 '경임이는 화장실로 갔습니다.',
 '수종이는 복도로 갔습니다.',
 '필웅이는 부엌으로 가버렸습니다.',
 '수종이는 부엌으로 이동했습니다.',
 '수종이는 화장실로 가버렸습니다.']

In [98]:
# Question, Answer 확인
i = 0
for q, a in zip(train_questions, train_answers):
    print(q, a)
    i+=1
    if i == 10:
        break

필웅이는 어디야?  화장실
수종이는 어디야?  복도
수종이는 어디야?  복도
수종이는 어디야?  사무실
경임이는 어디야?  화장실
경임이는 어디야?  화장실
경임이는 어디야?  화장실
경임이는 어디야?  화장실
은경이는 어디야?  사무실
수종이는 어디야?  복도


## Tokenizing and Preprocessing
* Tokenizer: https://github.com/lovit/customized_konlpy

In [99]:
from ckonlpy.tag import Twitter

# 없는 품사 태깅 추가
twitter = Twitter()
twitter.add_dictionary('은경이', 'Noun')

In [100]:
from ckonlpy.tag import Postprocessor

# 불용어 추가
stopwords = {'는', '은', '이', '가'}
postprocessor = Postprocessor(base_tagger = twitter, stopwords = stopwords)

In [101]:
print(postprocessor.pos('은경이는 사무실로 갔습니다.'))
print(twitter.morphs('은경이는 사무실로 갔습니다.'))

[('은경이', 'Noun'), ('사무실', 'Noun'), ('로', 'Josa'), ('갔습니다', 'Verb'), ('.', 'Punctuation')]
['은경이', '는', '사무실', '로', '갔습니다', '.']


In [102]:
print(postprocessor.pos('수종이는 어디야?'))
print(twitter.morphs('수종이는 어디야?'))

[('수종', 'Noun'), ('어디', 'Noun'), ('야', 'Josa'), ('?', 'Punctuation')]
['수종', '이', '는', '어디', '야', '?']


=> 은경이는 단어사전에 추가되어 '은경이'를 하나의 형태소로 구분한다. 그러나, 수종이는 단어사전에 추가하지 않았기 때문에 '이'는 불용어로 구분된다.

In [103]:
# train_questions에 있는 이름 추가
arr = []

for line in train_questions:
    arr.append(line.split('는')[0])

In [104]:
list(set(arr))

['수종이', '은경이', '경임이', '필웅이']

In [105]:
# test_questions에 있는 이름 추가
arr = []

for line in test_questions:
    arr.append(line.split('는')[0])

In [106]:
name_list = list(set(arr))

In [107]:
for name in name_list:
    twitter.add_dictionary(name, 'Noun') 

In [108]:
print(postprocessor.pos('수종이는 어디야?'))
print(twitter.morphs('수종이는 어디야?'))

[('수종이', 'Noun'), ('어디', 'Noun'), ('야', 'Josa'), ('?', 'Punctuation')]
['수종이', '는', '어디', '야', '?']


In [17]:
def tokenize(sent):
    return [ x.strip() for x in re.sub(r"\s+|\b", '\f', sent).split('\f') if x.strip() ] # python 3.7의 경우 
    # return [ x.strip() for x in re.split('(\W+)?', sent) if x.strip()] # python 3.6의 경우

In [35]:
def preprocess_data(train_data, test_data):
    counter = FreqDist()
    
    # 두 문장의 story를 하나의 문장으로 통합하는 함수
    flatten = lambda data: reduce(lambda x, y: x + y, data)

    # 각 샘플의 길이를 저장하는 리스트
    story_len = []
    question_len = []
    
    for stories, questions, answers in [train_data, test_data]:
        for story in stories:
            stories = tokenize(flatten(story)) # 스토리의 문장들을 펼친 후 토큰화
            story_len.append(len(stories)) # 각 story의 길이 저장
            for word in stories: # 단어 집합에 단어 추가
                counter[word] += 1
        for question in questions:
            question = tokenize(question)
            question_len.append(len(question))
            for word in question:
                counter[word] += 1
        for answer in answers:
            answer = tokenize(answer)
            for word in answer:
                counter[word] += 1

    # 단어장 생성
    word2idx = {word : (idx + 1) for idx, (word, _) in enumerate(counter.most_common())}
    idx2word = {idx : word for word, idx in word2idx.items()}

    # 가장 긴 샘플의 길이
    story_max_len = np.max(story_len)
    question_max_len = np.max(question_len)

    return word2idx, idx2word, story_max_len, question_max_len

In [36]:
word2idx, idx2word, story_max_len, question_max_len = preprocess_data(train_data, test_data)

In [37]:
print(word2idx)

{'.': 1, '경임이는': 2, '은경이는': 3, '수종이는': 4, '필웅이는': 5, '이동했습니다': 6, '가버렸습니다': 7, '뛰어갔습니다': 8, '복귀했습니다': 9, '갔습니다': 10, '화장실로': 11, '정원으로': 12, '복도로': 13, '어디야': 14, '?': 15, '부엌으로': 16, '사무실로': 17, '침실로': 18, '화장실': 19, '정원': 20, '사무실': 21, '침실': 22, '복도': 23, '부엌': 24}


In [38]:
print(idx2word)

{1: '.', 2: '경임이는', 3: '은경이는', 4: '수종이는', 5: '필웅이는', 6: '이동했습니다', 7: '가버렸습니다', 8: '뛰어갔습니다', 9: '복귀했습니다', 10: '갔습니다', 11: '화장실로', 12: '정원으로', 13: '복도로', 14: '어디야', 15: '?', 16: '부엌으로', 17: '사무실로', 18: '침실로', 19: '화장실', 20: '정원', 21: '사무실', 22: '침실', 23: '복도', 24: '부엌'}


In [22]:
vocab_size = len(word2idx) + 1

In [23]:
print('* 단어사전의 크기: ',vocab_size)
print('* 스토리의 최대 길이: ',story_max_len)
print('* 질문의 최대 길이: ',question_max_len)

* 단어사전의 크기:  25
* 스토리의 최대 길이:  40
* 질문의 최대 길이:  3


## Vectorization

In [24]:
def vectorize(data, word2idx, story_maxlen, question_maxlen):
    Xs, Xq, Y = [], [], []
    flatten = lambda data: reduce(lambda x, y: x + y, data)

    stories, questions, answers = data
    for story, question, answer in zip(stories, questions, answers):
        xs = [word2idx[w] for w in tokenize(flatten(story))]
        xq = [word2idx[w] for w in tokenize(question)]
        Xs.append(xs)
        Xq.append(xq)
        Y.append(word2idx[answer])

    # 스토리와 질문은 각각의 최대 길이로 패딩
    # 정답은 원-핫 인코딩
    return pad_sequences(Xs, maxlen=story_maxlen),\
           pad_sequences(Xq, maxlen=question_maxlen),\
           to_categorical(Y, num_classes=len(word2idx) + 1)

In [25]:
Xstrain, Xqtrain, Ytrain = vectorize(train_data, word2idx, story_max_len, question_max_len)
Xstest, Xqtest, Ytest = vectorize(test_data, word2idx, story_max_len, question_max_len)

In [26]:
print(Xstrain.shape, Xqtrain.shape, Ytrain.shape, Xstest.shape, Xqtest.shape, Ytest.shape)

(10000, 40) (10000, 3) (10000, 25) (1000, 40) (1000, 3) (1000, 25)


## Model Train

In [27]:
# 에포크 횟수
train_epochs = 120

# 배치 크기
batch_size = 32

# 임베딩 크기
embed_size = 50

# LSTM의 크기
lstm_size = 64

# 과적합 방지 기법인 드롭아웃 적용 비율
dropout_rate = 0.30

In [28]:
input_sequence = Input((story_max_len,))
question = Input((question_max_len,))
 
print('Stories :', input_sequence)
print('Question:', question)

Stories : KerasTensor(type_spec=TensorSpec(shape=(None, 40), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
Question: KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")


In [29]:
# 스토리를 위한 첫 번째 임베딩. 그림에서의 Embedding A
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=embed_size))
input_encoder_m.add(Dropout(dropout_rate))
# 결과 : (samples, story_max_len, embed_size) / 샘플의 수, 문장의 최대 길이, 임베딩 벡터의 차원

In [30]:
# 스토리를 위한 두 번째 임베딩. 그림에서의 Embedding C
# 임베딩 벡터의 차원을 question_max_len(질문의 최대 길이)로 한다.
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=question_max_len))
input_encoder_c.add(Dropout(dropout_rate))
# 결과 : (samples, story_max_len, question_max_len) / 샘플의 수, 문장의 최대 길이, 질문의 최대 길이(임베딩 벡터의 차원)

In [31]:
# 질문을 위한 임베딩. 그림에서의 Embedding B
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=embed_size,
                               input_length=question_max_len))
question_encoder.add(Dropout(dropout_rate))
# 결과 : (samples, question_max_len, embed_size) / 샘플의 수, 질문의 최대 길이, 임베딩 벡터의 차원

In [32]:
# 실질적인 임베딩 과정
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

print('Input encoded m', input_encoded_m, '\n')
print('Input encoded c', input_encoded_c, '\n')
print('Question encoded', question_encoded, '\n')

Input encoded m KerasTensor(type_spec=TensorSpec(shape=(None, 40, 50), dtype=tf.float32, name=None), name='sequential/dropout_20/Identity:0', description="created by layer 'sequential'") 

Input encoded c KerasTensor(type_spec=TensorSpec(shape=(None, 40, 3), dtype=tf.float32, name=None), name='sequential_1/dropout_21/Identity:0', description="created by layer 'sequential_1'") 

Question encoded KerasTensor(type_spec=TensorSpec(shape=(None, 3, 50), dtype=tf.float32, name=None), name='sequential_2/dropout_22/Identity:0', description="created by layer 'sequential_2'") 



In [39]:
# 스토리 단어들과 질문 단어들 간의 유사도를 구하는 과정
# 유사도는 내적을 사용한다.
match = dot([input_encoded_m, question_encoded], axes=-1, normalize=False)
match = Activation('softmax')(match)
print('Match shape', match)
# 결과 : (samples, story_max_len, question_max_len) / 샘플의 수, 문장의 최대 길이, 질문의 최대 길이

Match shape KerasTensor(type_spec=TensorSpec(shape=(None, 40, 3), dtype=tf.float32, name=None), name='activation/truediv:0', description="created by layer 'activation'")


In [40]:
# 매칭 유사도 행렬과 질문에 대한 임베딩을 더한다.
response = add([match, input_encoded_c])  # (samples, story_maxlen, question_max_len)
response = Permute((2, 1))(response)  # (samples, question_max_len, story_maxlen)
print('Response shape', response)

Response shape KerasTensor(type_spec=TensorSpec(shape=(None, 3, 40), dtype=tf.float32, name=None), name='permute/transpose:0', description="created by layer 'permute'")


In [55]:
# concatenate the response vector with the question vector sequence
answer = concatenate([response, question_encoded])
print('Answer shape', answer.shape)
print(type(answer))

Answer shape (None, 3, 90)
<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>


In [57]:
import tensorflow as tf

tf.keras.backend.is_keras_tensor(answer)

True

In [46]:
generate_answer = Sequential()
generate_answer.add(LSTM(lstm_size))  # Generate tensors of shape 32
generate_answer.add(Dropout(dropout_rate))
generate_answer.add(Dense(vocab_size))  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
generate_answer.add(Activation('softmax'))

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [58]:
answer = LSTM(lstm_size, input_shape = answer.shape)(answer)  # Generate tensors of shape 32
answer = Dropout(dropout_rate)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported

In [ ]:
# 모델 컴파일
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['acc'])
 
# 테스트 데이터를 검증 데이터로 사용하면서 모델 훈련 시작
history = model.fit([Xstrain, Xqtrain],
         Ytrain, batch_size, train_epochs,
         validation_data=([Xstest, Xqtest], Ytest))
 
# 훈련 후에는 모델 저장
model_path = os.getenv('HOME')+'/aiffel/babi_memory_net/model.h5'
model.save(model_path)

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate([Xstest, Xqtest], Ytest)[1]))

## Visualization

In [ ]:
# plot accuracy and loss plot
plt.subplot(211)
plt.title("Accuracy")
plt.plot(history.history["acc"], color="g", label="train")
plt.plot(history.history["val_acc"], color="b", label="validation")
plt.legend(loc="best")

plt.subplot(212)
plt.title("Loss")
plt.plot(history.history["loss"], color="g", label="train")
plt.plot(history.history["val_loss"], color="b", label="validation")
plt.legend(loc="best")

plt.tight_layout()
plt.show()

# labels
ytest = np.argmax(Ytest, axis=1)

# get predictions
Ytest_ = model.predict([Xstest, Xqtest])
ytest_ = np.argmax(Ytest_, axis=1)

In [ ]:
NUM_DISPLAY = 30

print("{:20}|{:7}|{}".format("질문", "실제값", "예측값"))
print(39 * "-")

for i in range(NUM_DISPLAY):
    question = " ".join([idx2word[x] for x in Xqtest[i].tolist()])
    label = idx2word[ytest[i]]
    prediction = idx2word[ytest_[i]]
    print("{:20}: {:8} {}".format(question, label, prediction))

## Result
|평가문항|상세기준|
|------|------|
|1. 한국어의 특성에 알맞게 전처리가 진행되었다.|한국어 특성에 따른 토큰화, 임베딩을 거쳐 데이터셋이 적절히 구성되었다.
|2. 메모리 네트워크가 정상적으로 구현되어 학습이 안정적으로 진행되었다.|validation loss가 안정적으로 수렴하는 것을 확인하고 이를 시각화하였다.
|
|3. 메모리 네트워크를 통해 한국어 bAbI 태스크의 높은 정확도를 달성하였다.|추론 태스크의 테스트 정확도가 90% 이상 달성하였다.|